# Recognition pipeline

In [ ]:
%load_ext autoreload
%autoreload 2

In [58]:
import PIL.Image
import pandas as pd
import cv2
from math import floor
import numpy as np
import os
import matplotlib.pyplot as plt
from recognition_functions import *
from plot_helper import *
from table_extractor import *
from chip_recognition import *
from collections import defaultdict

In [ ]:
path_data = "data/train"
path_setup = "data/image_setup/"

file = os.path.join(path_data, f"train_{str(0).zfill(2)}.jpg")
file_xtrem = os.path.join(path_setup, "ultimate_test.jpg")

# open the image
im_xtrem = PIL.Image.open(file_xtrem)
plt.imshow(im_xtrem)
plt.show()

# open the image
im = PIL.Image.open(file)
plt.imshow(im)
plt.show()

### Read images

In [ ]:
images = []

for i in range(28):
    #if i not in problematic_imgs:
    file = os.path.join(path_data, f"train_{str(i).zfill(2)}.jpg")
    im = PIL.Image.open(file)
    images.append(np.array(im))

In [ ]:
plt.imshow(images[0])
plt.show()

### Extract table

In [ ]:
table_imgs = []
for img in images:
    table_img = TableExtractor.extract_table(img,plot=False)
    table_imgs.append(table_img)

### Partition image

In [ ]:
partitions = []
for table_img in table_imgs:
    partition = partition_image(table_img)
    partitions.append(partition)

### Manual Treshold Fine-tuning

In [ ]:
#Create overall image by concatenating 25 out of 28 partitions
chips_all = None
for i in range(5):
    chips_row = None
    for j in range(5):
        chips = partitions[i*5+j]['chips']
        if(type(chips_row)==type(None)):
            chips_row = chips
        else:
            chips_row = np.concatenate([chips_row,chips],axis=1)
    if(type(chips_all)==type(None)):
        chips_all = chips_row
    else:
        chips_all = np.concatenate([chips_all,chips_row],axis=0)

    chips_row = None

chips_all = cv2.medianBlur(chips_all,ksize=51)
plt.imshow(chips_all)
plt.show()

chips_reduced = cv2.resize(chips_all, (2000,2000), interpolation = cv2.INTER_AREA)
plt.imshow(chips_reduced)
plt.show()

In [ ]:
from chip_recognition import *

chips_rgb = chips_reduced

window_treshold(chips_rgb,HSV_COLOR_BOUNDS,color='blue')

### Count chips

In [62]:
#Compute accuracy
labels = pd.read_csv('data/train/train_labels.csv')
color_symbols = ['R','G','B','K','W']
symbol_precisions = defaultdict(list)
for (index,row), partition in zip(labels.iterrows(),partitions):
    chips = partition['chips']
    chip_counts = count_chips(chips,plot=False)
    for symbol,count in chip_counts.items():
        correct_count = row[symbol]
        print(f"Predicted {count} chips of {symbol} actual is {correct_count}")
        #precision = abs(count - correct_count)

Detected 1 tokens of color red
Detected 1 tokens of color green
Detected 3 tokens of color blue
Predicted 1 chips of R actual is 1
Predicted 1 chips of G actual is 1
Predicted 3 chips of B actual is 1
Detected 2 tokens of color red
Detected 1 tokens of color green
Detected 1 tokens of color blue
Predicted 2 chips of R actual is 2
Predicted 1 chips of G actual is 0
Predicted 1 chips of B actual is 1
Detected 0 tokens of color red
Detected 2 tokens of color green
Detected 9 tokens of color blue
Predicted 0 chips of R actual is 0
Predicted 2 chips of G actual is 0
Predicted 9 chips of B actual is 8
Detected 0 tokens of color red
Detected 6 tokens of color green
Detected 4 tokens of color blue
Predicted 0 chips of R actual is 0
Predicted 6 chips of G actual is 2
Predicted 4 chips of B actual is 4
Detected 2 tokens of color red
Detected 1 tokens of color green
Detected 1 tokens of color blue
Predicted 2 chips of R actual is 3
Predicted 1 chips of G actual is 1
Predicted 1 chips of B actual 